In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [9]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [10]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.experiment_interface import run_exp_iter_with_LFR

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [11]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'LFR_acs_income'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [{'k': 10, 'Ax': 0.1, 'Ay': 0.9, 'Az': 2.0}]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp', 'LFR', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                'LFR', EXPERIMENT_NAME, 'folk_GA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [12]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [13]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [14]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
    # 'session_uuid': '',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  114a1071-264b-49e6-b298-23db4031e9c2


## Initialize custom objects

In [15]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [16]:
data_loader.X_data.shape

(15000, 10)

## Run experiment iterations

### Experiment iteration 1

In [24]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [25]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [1]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      with_tuning=True,
                      # with_tuning=False,
                      # tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

### Experiment iteration 2

In [21]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [2]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

### Experiment iteration 3

### Experiment iteration 4

### Experiment iteration 5

### Experiment iteration 6